In [18]:
import os
import cv2
import numpy as np

# Define dataset directory (Kaggle dataset path)
data_dir = "/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"
classes = ['collie', 'dolphin', 'elephant', 'fox', 'moose', 'rabbit', 'sheep', 'squirrel', 'giant+panda', 'polar+bear']

# Initialize lists for data and labels
X_data = []
y_data = []

# Loop through each class to load images
for label, class_name in enumerate(classes):
    class_path = os.path.join(data_dir, class_name)
    images = os.listdir(class_path)[:650]  # Load only the first 650 images per class
    for img_name in images:
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))  # Resize images to 128x128
        X_data.append(img)
        y_data.append(label)

# Convert lists to NumPy arrays
X_data = np.array(X_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.int32)

# Verify shapes
print(f"X_data shape: {X_data.shape}")  
print(f"y_data shape: {y_data.shape}")  

# Load Original Dataset 

X_data shape: (6500, 128, 128, 3)
y_data shape: (6500,)


# Split Dataset into Train and Test

In [19]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

# Verify shapes
print(f"X_train shape: {X_train.shape}") 
print(f"y_train shape: {y_train.shape}")  
print(f"X_test shape: {X_test.shape}")    
print(f"y_test shape: {y_test.shape}")    

X_train shape: (4550, 128, 128, 3)
y_train shape: (4550,)
X_test shape: (1950, 128, 128, 3)
y_test shape: (1950,)


# Perform Data Augmentation 

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation generator
datagen = ImageDataGenerator(rotation_range=20)  # Set rotation range for augmentation

# Initialize lists for augmented data and labels
augmented_data = []
augmented_labels = []

for i in range(len(X_train)):
    image = X_train[i]
    label = y_train[i]

    # Add original image
    if image.shape == (128, 128):  # Convert grayscale to RGB if necessary
        image = np.stack((image,) * 3, axis=-1)
    elif image.shape == (128, 3):  # Incorrect shape
        image = np.expand_dims(image, axis=1)
        image = np.tile(image, (1, 128, 1))
    augmented_data.append(image)
    augmented_labels.append(label)

    # Apply Gaussian blur
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    if blurred_image.shape == (128, 128):  # Convert grayscale to RGB if necessary
        blurred_image = np.stack((blurred_image,) * 3, axis=-1)
    elif blurred_image.shape == (128, 3):  # Incorrect shape
        blurred_image = np.expand_dims(blurred_image, axis=1)
        blurred_image = np.tile(blurred_image, (1, 128, 1))
    augmented_data.append(blurred_image)
    augmented_labels.append(label)

    # Apply rotation using ImageDataGenerator
    for batch in datagen.flow(np.expand_dims(image, axis=0), batch_size=1, shuffle=False):
        rotated_image = batch[0][0]
        if rotated_image.shape == (128, 128):
            rotated_image = np.stack((rotated_image,) * 3, axis=-1)
        elif rotated_image.shape == (128, 3):
            rotated_image = np.expand_dims(rotated_image, axis=1)
            rotated_image = np.tile(rotated_image, (1, 128, 1))
        augmented_data.append(rotated_image)
        augmented_labels.append(label)
        break  # Only take one rotated image per original

# Convert augmented data to NumPy arrays
augmented_data = np.array(augmented_data, dtype=np.float32)
augmented_labels = np.array(augmented_labels, dtype=np.int32)

# Verify final shapes
print(f"Final augmented_data shape: {augmented_data.shape}")
print(f"Final augmented_labels shape: {augmented_labels.shape}")

Final augmented_data shape: (13650, 128, 128, 3)
Final augmented_labels shape: (13650,)


# One-Hot Encode Labels

In [21]:
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoded format
y_train_categorical = to_categorical(augmented_labels, num_classes=10)
y_test_categorical = to_categorical(y_test, num_classes=10)

# Verify shapes
print(f"y_train_categorical shape: {y_train_categorical.shape}") 
print(f"y_test_categorical shape: {y_test_categorical.shape}")  

y_train_categorical shape: (13650, 10)
y_test_categorical shape: (1950, 10)


# Design CNN Model

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Define the CNN model
model = Sequential()

# Add the Input layer explicitly
model.add(Input(shape=(128, 128, 3)))  # Define input shape

# First convolutional block
model.add(Conv2D(32, (3, 3), activation='relu'))  # First convolutional layer
model.add(MaxPooling2D(pool_size=(2, 2)))  # First pooling layer

# Second convolutional block
model.add(Conv2D(64, (3, 3), activation='relu'))  # Second convolutional layer
model.add(MaxPooling2D(pool_size=(2, 2)))  # Second pooling layer

# Third convolutional block
model.add(Conv2D(128, (3, 3), activation='relu'))  # Third convolutional layer
model.add(MaxPooling2D(pool_size=(2, 2)))  # Third pooling layer

# Flatten the feature maps
model.add(Flatten())

# Fully connected (dense) layers
model.add(Dense(256, activation='relu'))  # Fully connected layer with increased neurons
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(10, activation='softmax'))  # Output layer for 10 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,518,602 (24.87 MB)

 Trainable params: 6,518,602 (24.87 MB)

 Non-trainable params: 0 (0.00 B)

# Train the Model

In [24]:
history = model.fit(
    augmented_data,                 
    y_train_categorical,            
    epochs=10,                      
    batch_size=32,                  
    validation_data=(X_test, y_test_categorical),  
    verbose=1                       
)


Epoch 1/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 259s 600ms/step - accuracy: 0.1788 - loss: 9.6676 - val_accuracy: 0.2154 - val_loss: 2.1292
Epoch 2/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 253s 592ms/step - accuracy: 0.2404 - loss: 2.0689 - val_accuracy: 0.2641 - val_loss: 2.1262
Epoch 3/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 272s 636ms/step - accuracy: 0.3283 - loss: 1.8593 - val_accuracy: 0.2733 - val_loss: 2.2588
Epoch 4/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 259s 606ms/step - accuracy: 0.3997 - loss: 1.6895 - val_accuracy: 0.2779 - val_loss: 2.4745
Epoch 5/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 259s 607ms/step - accuracy: 0.4585 - loss: 1.5297 - val_accuracy: 0.2605 - val_loss: 2.8606
Epoch 6/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 258s 605ms/step - accuracy: 0.4758 - loss: 1.4869 - val_accuracy: 0.3164 - val_loss: 2.7713
Epoch 7/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 258s 605ms/step - accuracy: 0.5373 - loss: 1.2991 - val_accuracy: 0.3369 - val_loss: 2.9791
Epoch 8/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 269s 631ms/step - accuracy: 0.5817 -

In [25]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical, verbose=1)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


61/61 ━━━━━━━━━━━━━━━━━━━━ 10s 162ms/step - accuracy: 0.3588 - loss: 3.9002
Test Loss: 3.980386972427368
Test Accuracy: 0.3528205156326294
